In [2]:
import pyspark
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master('local')\
    .appName('myApp')\
    .config(conf = myConf)\
    .getOrCreate()

21/12/18 09:45:34 WARN Utils: Your hostname, Kritiasui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 172.30.1.23 instead (on interface en0)
21/12/18 09:45:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/12/18 09:45:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [1]:
# data : RDD로 만든 뒤 ','로 스플릿해서 DF로 변환한다.
marks = ['김하나, English, 100',
         '김하나, Math, 80',
         '임하나, English, 70',
         '임하나, Math, 100',
         '김갑돌, English, 82.3',
         '김갑돌, Math, 98.5']


In [5]:
# RDD 생성 및 스플릿
_marksRdd = spark.sparkContext.parallelize(marks).map(lambda x: x.split(','))
_marksDf = spark.createDataFrame(_marksRdd, schema=['name', 'subject', 'mark'])
_marksDf.printSchema()
_marksDf.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)

+------+--------+-----+
|  name| subject| mark|
+------+--------+-----+
|김하나| English|  100|
|김하나|    Math|   80|
|임하나| English|   70|
|임하나|    Math|  100|
|김갑돌| English| 82.3|
|김갑돌|    Math| 98.5|
+------+--------+-----+



In [7]:
_marksDf.groupby('subject').count().show()

+--------+-----+
| subject|count|
+--------+-----+
|    Math|    3|
| English|    3|
+--------+-----+



# 그룹 윈도우 함수

In [8]:
from pyspark.sql.window import Window
win = Window.partitionBy('subject').orderBy('mark')